## Build Data Notebook

This notebook performs the following steps:

- **Process the `bam_file`** for each dataset to generate a **counts DataFrame**.
- **Construct AnnData matrices** from the counts DataFrame, representing the naive counts for \( u[k] \) across all \( k \).
- **Generate predicted counts** using:
  - The **uniform model**
  - The **non-uniform model**


### 1. Processing the Bam_file

In [1]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy
import sklearn
import anndata as ad 
from tqdm import tqdm as tqdm
import pysam
import scanpy as sc

In [ ]:
import pysam
import pandas as pd

# 1) INPUT: Path to the filtered BAM file (choose one dataset)
filtered_bam_path = '/data/dagyeman/cellranger/bam_file_analysis/1k_PBMCs/filtered_1k_PBMCS_bam.bam'   # BAM file for 1k dataset
# filtered_bam_path = '/data/dagyeman/cellranger/bam_file_analysis/10k_PBMCs/filtered_10k_PBMCS_bam.bam' # BAM file for 10k dataset
# filtered_bam_path = '/data/dagyeman/cellranger/bam_file_analysis/500_PBMCs/filtered_500_PBMCS_bam.bam'    # BAM file for 500 dataset
# filtered_bam_path = '/data/dagyeman/cellranger/bam_file_analysis/5k_PBMCs/filtered_5k_PBMCS_bam.bam'    # BAM file for 5k dataset

# 2) data: list to hold extracted (barcode, gene, UMI) triplets
data = []

# 3) EXTRACT: Read BAM file and pull out barcode, gene, and UMI tags
with pysam.AlignmentFile(filtered_bam_path, "rb") as bam_file:
    for read in bam_file:
        if read.has_tag('CB') and read.has_tag('GN') and read.has_tag('UB'):
            barcode = read.get_tag('CB')  # string: cell barcode
            gene = read.get_tag('GN')     # string: gene name
            umi = read.get_tag('UB')      # string: unique molecular identifier
            data.append([barcode, gene, umi])

# 4) df: pandas DataFrame containing all reads with columns (barcode, gene, UMI)
df = pd.DataFrame(data, columns=['barcode', 'gene', 'UMI'])
print("First few raw rows:")
print(df.head())

# 5) CLEAN: Remove any rows where UMI contains ambiguous base 'N'
df = df[~df['UMI'].str.contains('N')]

# 6) deduplicated_df: DataFrame containing only unique (barcode, gene, UMI) combinations
#    This is the deduplicated set of observed UMIs before truncating
deduplicated_df = df.drop_duplicates(subset=['barcode', 'gene', 'UMI']).reset_index(drop=True)
print("First few deduplicated rows:")
print(deduplicated_df.head())

# 7) LOOP: For UMI lengths k = 1 to 12, compute number of unique UMIs per (barcode, gene)
for k in range(1, 13):
    deduplicated_df[f'UMI_{k}'] = deduplicated_df['UMI'].str[:k]  # first k bases
    grouped_df = (
        deduplicated_df.groupby(['barcode', 'gene'])[f'UMI_{k}']
                       .nunique()
                       .reset_index()
                       .rename(columns={f'UMI_{k}': f'unique_UMI_count_{k}'})
    )
    print(f"Unique UMI counts for UMI length {k}:")
    print(grouped_df.head())

# 8) final_df: merged table with unique UMI counts for all lengths 1–12
final_df = (
    deduplicated_df.groupby(['barcode', 'gene'])
                   .agg({f'UMI_{k}': 'nunique' for k in range(1, 13)})
                   .reset_index()
)
final_df.columns = ['barcode', 'gene'] + [f'unique_UMI_count_{k}' for k in range(1, 13)]
print("Final merged table:")
print(final_df.head())


### 2. Adata Objects: Naive Method - created from final_df dataframe

In [ ]:
import pandas as pd
import anndata as ad

# Dictionary to store the AnnData objects for each UMI length
adata_dict = {}

# Loop over UMI lengths from 1 to 12
for k in range(1, 13):
    # Extract the relevant columns for the current UMI length
    # Contains: barcode, gene, and the deduplicated count for UMI length k
    adata_matrix = final_df[['barcode', 'gene', f'unique_UMI_count_{k}']]

    # Pivot the DataFrame:
    # - Rows = barcodes (cells)
    # - Columns = genes
    # - Values = deduplicated UMI counts for the current UMI length
    matrix_df = adata_matrix.pivot(
        index='barcode',
        columns='gene',
        values=f'unique_UMI_count_{k}'
    ).fillna(0)

    # Create an AnnData object from the counts matrix
    adata = ad.AnnData(X=matrix_df.values)

    # Assign observation names (rows) to the barcodes
    adata.obs_names = matrix_df.index

    # Assign variable names (columns) to the gene names
    adata.var_names = matrix_df.columns

    # Store the AnnData object in the dictionary with the UMI length as the key
    adata_dict[k] = adata

# Example: View the AnnData object for UMI length 1
print(adata_dict[1])


### 3. Adata Objects: Uniform Estimator 

#### Model

In [ ]:
def mom_estimator_unif(y, K):
    """
    Method-of-moments estimator for the uniform UMI model.

    Given observed unique molecule counts y and total possible UMIs K,
    estimates the underlying molecule counts n using the closed-form
    inversion for the uniform collision model.

    Works element-wise on scalars, vectors, or matrices.
    Special handling is applied when y == K using a recursive relation.
    """
    # Make sure y is a NumPy array
    y = np.array(y)
    
    # Prepare an output array of floats with the same shape as y
    result = np.empty_like(y, dtype=float)
    
    # Handle entries where y equals K
    special_mask = (y == K)
    if special_mask.any():
        # Special case value from recursive formula
        special_value = mom_estimator_unif(K - 1, K) + K
        result[special_mask] = special_value
    
    # Handle all other entries
    general_mask = ~special_mask
    if general_mask.any():
        denominator = np.log(1 - 1/K)
        result[general_mask] = np.log(1 - y[general_mask] / K) / denominator
    
    return result

#### Generating AnnData matrix for uniform estimator at length k

In [ ]:
import scanpy as sc
import anndata as ad
import pandas as pd

filepath = "/data/dagyeman/cellranger/bam_file_analysis/1k_PBMCs/ub_objects" #1k
# filepath = "/data/dagyeman/cellranger/bam_file_analysis/10k_PBMCs/ub_objects" #10k

# Load naive AnnData counts (UMI lengths 1–12)
adata_dict = {}
for i in range(1, 13):
    adata_dict[i] = sc.read_h5ad(f"{filepath}/adata_matrices/adata_{i}.h5ad")
    

# Build the predicted AnnData for a chosen UMI length k 
chosen_k = 6  # set this as needed

adata = adata_dict[chosen_k]                 # naive counts AnnData for k

# Dense matrix view of counts
matrix = adata.X.toarray() if hasattr(adata.X, "toarray") else adata.X
original_matrix = matrix.copy()              # keep a copy if you want to compare later

# Predict counts with your estimator
predicted_matrix = mom_estimator_unif(matrix, 4**chosen_k)
# AnnData with predicted counts (cells x genes align with the naive object)
predicted_adata = ad.AnnData(X=predicted_matrix)
predicted_adata.obs_names = adata.obs_names
predicted_adata.var_names = adata.var_names

# Example check
print(predicted_adata)

### 4. Adata Objects: Non-uniform Model

#### Function

In [ ]:
import numpy as np
from numba import njit, prange

@njit
def interpolate(y, f_hat_values, n_values):
    """Interpolates the value of n for a given y using linear interpolation."""
    if y == 0:
        return 0
    # Find index in f_hat_values where y fits in
    i = np.searchsorted(f_hat_values, y) - 1
    # Clamp i so it’s never out of range
    i = max(0, min(i, len(f_hat_values) - 2))

    # Values for n and f at positions i and i+1
    n_i = n_values[i]
    n_ip1 = n_values[i + 1]
    f_ni = f_hat_values[i]
    f_nip1 = f_hat_values[i + 1]

    # Linear interpolation
    denom = f_nip1 - f_ni
    return n_i + (y - f_ni) * (n_ip1 - n_i) / denom

@njit(parallel=True)
def interpolate_array(matrix_flat, f_hat_values, n_values, j):
    """Applies interpolation to each element in a flattened matrix."""
    result = np.empty_like(matrix_flat)
    target_value = 4 ** j

    for idx in prange(matrix_flat.size):
        y = matrix_flat[idx]
        if y == target_value:
            # Special handling if value equals theoretical max
            result[idx] = (
                3 * interpolate(y - 1, f_hat_values, n_values)
                - 3 * interpolate(y - 2, f_hat_values, n_values)
                + interpolate(y - 3, f_hat_values, n_values)
            )
        else:
            result[idx] = interpolate(y, f_hat_values, n_values)
    return result

def mom_estimator_nonunif(prob_arr, matrix, j, n_max=15000):
    """Returns the inverted matrix using the method of moments."""
    # Grid of possible n values
    n_values = np.linspace(1, n_max, n_max)
    # Expected f-hat for each n
    f_hat_values = np.array([4**j - np.sum((1 - prob_arr)**n) for n in n_values])

    # Flatten matrix for easier processing
    matrix_flat = matrix.flatten().astype(np.float64)

    # Check for out-of-bounds y values (allow exactly 4**j)
    f_min, f_max = f_hat_values[0], f_hat_values[-1]
    target = 4**j
    mask = (matrix_flat > f_max) & (matrix_flat != target)
    out_of_bounds = matrix_flat[mask]
    if out_of_bounds.size > 0:
        print(f"Out-of-bounds y-values: {out_of_bounds}")
        raise ValueError(f"Some y-values are outside interpolation range [{f_min}, {f_max}].")
        return "Failed to invert matrix due to out-of-bounds values."

    # Apply interpolation and reshape back to original
    interpolated_flat = interpolate_array(matrix_flat, f_hat_values, n_values, j)
    return interpolated_flat.reshape(matrix.shape)

#### Generating AnnData matrix for non-uniform estimator at length k


In [ ]:
import scanpy as sc
import anndata as ad
import pandas as pd

filepath = "/data/dagyeman/cellranger/bam_file_analysis/1k_PBMCs/ub_objects" #1k
# filepath = "/data/dagyeman/cellranger/bam_file_analysis/10k_PBMCs/ub_objects" #10k

# Load naive AnnData counts (UMI lengths 1–12)
adata_dict = {}
for i in range(1, 13):
    adata_dict[i] = sc.read_h5ad(f"{filepath}/adata_matrices/adata_{i}.h5ad")

# Load non-uniform UMI probability distributions (UMI lengths 1–12)
umi_prob_dict = {}
for i in range(1, 13):
    umi_prob_dict[i] = pd.read_csv(f"/data/dagyeman/cellranger/umi_probs/umi_probs_{i}.csv")

# Build the predicted AnnData for a chosen UMI length k
chosen_k = 6  # set this as needed

adata = adata_dict[chosen_k]                 # naive counts AnnData for k
umi_probs = umi_prob_dict[chosen_k]['prob']  # non-uniform probs for k

# Dense matrix view of counts
matrix = adata.X.toarray() if hasattr(adata.X, "toarray") else adata.X
original_matrix = matrix.copy()              # keep a copy if you want to compare later

# Predict counts with your estimator
predicted_matrix = mom_estimator_nonunif(umi_probs.values, matrix, j=chosen_k)

# AnnData with predicted counts (cells x genes align with the naive object)
predicted_adata = ad.AnnData(X=predicted_matrix)
predicted_adata.obs_names = adata.obs_names
predicted_adata.var_names = adata.var_names

# Example check
print(predicted_adata)
